In [ ]:
# test camera
import cv2
import os

img_id = 0
img_root = 'saved'
cap = cv2.VideoCapture(1)
while True:
    ret, frame = cap.read()
    cv2.imshow('frame', frame)

    if cv2.waitKey(1) & 0xFF == ord('s'):
        save_path = os.path.join(img_root, 'img_' + str(img_id) + '.png')
        cv2.imwrite(save_path, frame)
        print('saved')
        img_id += 1
cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

In [1]:
# test jaka

# export LD_LIBRARY_PATH=/home/osboxes/Desktop/GeneralRobot/SDK
# run before running this script

import sys
import os
sys.path.append(os.path.join(os.path.dirname(__file__), './SDK'))

import jkrc    
import time    
#坐标系    
COORD_BASE  = 0    
COORD_JOINT = 1    
COORD_TOOL  = 2    
#运动模式    
ABS = 0    
INCR= 1    
robot = jkrc.RC("192.168.2.160")    
robot.login()    
robot.power_on()    
robot.enable_robot()    
robot.drag_mode_enable(True)    
ret = robot.is_in_drag_mode()    
print(ret)    
a = input()    
robot.drag_mode_enable(False)    
ret = robot.is_in_drag_mode()    
print(ret)    
robot.logout()


NameError: name '__file__' is not defined

In [ ]:
# test planning

import os
import sys
import numpy as np
import argparse
from termcolor import cprint

from pybullet_planning import BASE_LINK, RED, BLUE, GREEN
from pybullet_planning import load_pybullet, connect, wait_for_user, LockRenderer, has_gui, WorldSaver, HideOutput, \
    reset_simulation, disconnect, set_camera_pose, has_gui, set_camera, wait_for_duration, wait_if_gui, apply_alpha
from pybullet_planning import Pose, Point, Euler
from pybullet_planning import multiply, invert, get_distance
from pybullet_planning import create_obj, create_attachment, Attachment
from pybullet_planning import link_from_name, get_link_pose, get_moving_links, get_link_name, get_disabled_collisions, \
    get_body_body_disabled_collisions, has_link, are_links_adjacent
from pybullet_planning import get_num_joints, get_joint_names, get_movable_joints, set_joint_positions, joint_from_name, \
    joints_from_names, get_sample_fn, plan_joint_motion
from pybullet_planning import dump_world, set_pose
from pybullet_planning import get_collision_fn, get_floating_body_collision_fn, expand_links, create_box
from pybullet_planning import pairwise_collision, pairwise_collision_info, draw_collision_diagnosis, body_collision_info

HERE = os.path.dirname(__file__)
robot_path = os.path.join(HERE, 'minicobo_v14', 'urdf', 'minicobo.urdf')
workspace_path = os.path.join(HERE, 'pybullet_planning_tutorials','examples','data', 'mit_3-412_workspace', 'urdf', 'mit_3-412_workspace.urdf')

def jaka_demo():
    # * this will create the pybullet GUI
    # setting viewers=False will enter GUI-free mode
    connect(use_gui=True)
    cprint("Welcome to pybullet! <Ctrl+left mouse> to rotate, <Ctrl+middle mouse> to move the camera, <Ctrl+right mouse> to zoom", 'green')
    cprint('But the space is empty, let\'s load our robot!', 'yellow')
    # wait_for_user is your friend! It will pause the console, but having a separate thread keeping
    # the GUI running so you can rotate and see
    wait_for_user()

    robot = load_pybullet(robot_path, fixed_base=True)
    # workspace = load_pybullet(workspace_path, fixed_base=True)

    # this will print all the bodies' information in your console
    dump_world()
    cprint('You just loaded a robot, a workspace (with many static objects as its link, I modeled our good old MIT 3-412 shop here), '
            + 'and an end effector (it\'s inside the robot base now)', 'green')
    wait_for_user()

    if has_gui():
        camera_base_pt = (0,0,0)
        camera_pt = np.array(camera_base_pt) + np.array([1, -0.5, 0.5])
        set_camera_pose(tuple(camera_pt), camera_base_pt)
    
    # * each joint of the robot are assigned with an integer in pybullet
    ik_joints = get_movable_joints(robot)
    ik_joint_names = get_joint_names(robot, ik_joints)
    cprint('Joint {} \ncorresponds to:\n{}'.format(ik_joints, ik_joint_names), 'green')

    # * get a joint configuration sample function:
    # it separately sample each joint value within the feasible range
    sample_fn = get_sample_fn(robot, ik_joints)

    robot_start_conf = sample_fn()
    cprint(f"This is before updating pose: {robot_start_conf}", 'yellow')
    wait_for_user()
    # * set joint configuration, the robot's pose will be updated
    set_joint_positions(robot, ik_joints, robot_start_conf)
    cprint("This is after set joint pose: {}".format(robot_start_conf), 'green')
    wait_for_user()

    robot_end_conf = list(sample_fn())

    path = plan_joint_motion(robot, ik_joints, robot_end_conf, self_collisions=True)
    if path is None:
        cprint('no plan found', 'red')
    else:
        wait_for_user('a motion plan is found! Press enter to start simulating!')

    # adjusting this number will adjust the simulation speed
    time_step = 0.03
    for conf in path:
        set_joint_positions(robot, ik_joints, conf)
        wait_for_duration(time_step)
    
    wait_for_user('Simulation finished! Press enter to disconnect.')
    disconnect()

jaka_demo() 